In [1]:
pip freeze > visualisation_requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [5]:
data_df = pd.read_csv("Project_data_csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Project_data_csv'

In [ ]:
data_df.shape

In [ ]:
data_df.head()

In [ ]:
data_df['DLS Correct?'].value_counts()

In [ ]:
DLS_accuracy=225310/(225310+44913)

In [ ]:
DLS_accuracy

In [ ]:
data_df.info()

In [ ]:
remaining_onb = []
for index, row in data_df.iterrows():
    onb = (row['Remaining Overs']*6) + row['Remaining Balls']
    remaining_onb.append(onb)
remaining_onb_df = pd.DataFrame(remaining_onb)

In [ ]:
remaining_onb_df

Below - creating a column for who wins - team 1 (team who bowled second), team 2(team who batted second) or a tie.  Can define these as bowling team and batting team respectively because I'm only using second innings data.

In [ ]:
winner_no = []
for index, row in data_df.iterrows():
    #Team 2 wins
    if row['Winner'] == row['batting_team']:
        winner_no.append(3)
    #Team 1 wins
    elif row['Winner'] == row['bowling_team']:
        winner_no.append(1)
    #Tie
    else:
       winner_no.append(2)

In [ ]:
winner_no_df = pd.DataFrame(winner_no)

In [ ]:
winner_no_df.info()

In [ ]:
winner_no_df.rename(columns={0: 'Winner_num'}, inplace=True)

Turning the Team 1 win, Team 2 win and tie numbers into 3 categories (1,2,3 respectively)

In [ ]:
#winner_no_df['Winner_num'] = winner_no_df.Winner_num.astype('category')


In [ ]:
data_df['Winner_num'] = winner_no_df['Winner_num']

In [ ]:
data_df['Remaining overs and balls'] = remaining_onb_df[0]

In [ ]:
data_df.info()

Chose numerical values that feed into my hypothesis (wickets taken, team value, ground scores and remaining overs and balls feed into the prediction.)  Changed Winner into a numerical field for prediction as this will be the target variable.

In [ ]:
data_df = data_df[['Unnamed: 0', 'Wickets taken', 'Remaining Team Value', 'Remaining Average Team Value', 'Start Team Value', 'Ground RPO', 'Ground RPW', 'Winner_num', 'Remaining overs and balls', 'Remainder']]

In [ ]:
data_df.rename(columns={'Unnamed: 0': 'Row ID'}, inplace=True)

In [ ]:
data_df

In [ ]:
data_df.info()

In [ ]:
data_df.describe()

In [ ]:
data_df.hist(bins=50, figsize=(15, 15))
plt.show()

Comments: Different scales, normal distribution of team value, slightly skewed of ground rpo and rpw.  Remainder funny - should I do it in balls rather than remainder?  Might make more sense.

Wickets taken and team value seem to follow Gaussian distribution and have low standard deviation (2.19 and 2.53 respectively)
Turned remaining overs and balls into remainder number and std much lower for that(7.87 compared to 47.24)

The other attributes have low standard deviation too but ground rpo, ground rpw and 

In [ ]:
data_df['Winner_num'].value_counts()

In [ ]:
data_df.shape

In [ ]:
from zlib import crc32

def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
train_set, test_set = split_data_with_id_hash(data_df, 0.2, "Row ID")

In [ ]:
train_set.shape

In [ ]:
test_set.shape

In [ ]:
data_test = test_set.copy()

In [ ]:
data_test = data_test.reset_index(drop=True)

In [ ]:
data_train = train_set.copy()

In [ ]:

data_train = data_train.reset_index(drop=True)

In [ ]:
data_train.to_csv("data_train_csv")

In [ ]:
data_train

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["Wickets taken", "Remaining overs and balls", "Remaining Team Value", "Remaining Average Team Value", "Start Team Value", "Ground RPO", "Ground RPW", "Winner_num"]
scatter_matrix(data_train[attributes], figsize=(25, 25))
plt.show()

In [ ]:
for a in attributes:
    plt.figure(figsize = (5, 5))
    sns.violinplot(x='Winner_num',y=a,data=data_train)

From the boxplots you can see some near separation between wickets taken and whether team 1 or team 2 win, but a mixture for tie.

For Team Value you also see some separation between team 1 and team 2, though more overlap than for the boxplots.  Again, tie for mixture.

Aside from some outliers, Ground RPO, Ground RPW and remaining overs and balls don't seem to predict whether team 1 or team 2 wins.

In [ ]:
corr_matrix = data_train.corr(method = 'pearson')

In [ ]:
corr_matrix
corr_matrix['Winner_num']

In [ ]:
sns.set(rc={'figure.figsize':(12,10)})
new_cmap = sns.diverging_palette(120, 120, as_cmap=True)
sns.heatmap(corr_matrix, annot=True, cmap=new_cmap, vmin=-1, vmax=1)

In [ ]:
skew = data_train.skew(axis='rows')
print(skew)

In [ ]:
sdklfhlksdfjls

#Commentary on correlation

Best correlation with winner are wickets taken and team value.  Unsurprisingly there's strong correlation between ground RPO and RPW.  Strong correlation betwen team value and wickets taken, which also makes sense as the more wickets taken the lower the team value.  Should I really be looking at the higher team value to start with rather than continuing team value?  Probably a better predictor but also again is that unfair?  Will run the code again and check.

Pearson or 

Data Commentary in general

Number of rows, balance of the category, skew, separability for the boxplots.

Upsampling/ oversampling to sort out imbalance

https://www.analyticsvidhya.com/blog/2020/10/overcoming-class-imbalance-using-smote-techniques/


Scaling first

In [ ]:
y_train = data_train['Winner_num']

In [ ]:
X_train = data_train.drop('Winner_num', axis=1)

In [ ]:
pip install imblearn
#https://arxiv.org/abs/1106.1813

from imblearn.over_sampling import SMOTE
smote_auto = SMOTE(sampling_strategy='auto', random_state=7)
smote_min = SMOTE(sampling_strategy='minority', random_state=7)

In [ ]:
X_train_resampled_auto, y_train_resampled_auto = smote_auto.fit_resample(X_train, y_train)
X_train_resampled_min, y_train_resampled_min = smote_min.fit_resample(X_train, y_train)

In [ ]:
#Class distribution before SMOTE
data_df['Winner_num'].value_counts()

In [ ]:
y_train_resampled_auto.value_counts()

In [ ]:
y_train_resampled_min.value_counts()

In [ ]:
y_train_resampled_min.shape

In [ ]:
X_train_resampled_auto.shape

In [ ]:
y_train_resampled_auto = pd.DataFrame(y_train_resampled_auto)
y_train_resampled_auto.set_axis(['Winner_num'], axis='columns')

y_train_resampled_min = pd.DataFrame(y_train_resampled_min)
y_train_resampled_min.set_axis(['Winner_num'], axis='columns')

In [ ]:
data_train_SMOTEauto = X_train_resampled_auto.copy()
data_train_SMOTEauto['Winner_num'] = y_train_resampled_auto

In [ ]:
data_train_SMOTEmin = X_train_resampled_min.copy()
data_train_SMOTEmin['Winner_num'] = y_train_resampled_min

In [ ]:
data_train_SMOTEmin.shape

In [ ]:
data_train_SMOTEauto.shape

In [ ]:
data_train_SMOTEauto.to_csv("data_train_SMOTEauto_csv")
data_train_SMOTEmin.to_csv("data_train_SMOTEmin_csv")

In [ ]:
data_train

In [ ]:
#x+1 because there are some 0 value
def normal(data):
    data_norm = data[['Wickets taken', 'Remaining Average Team Value', 'Start Team Value', 'Ground RPO', 'Ground RPW', 'Remaining overs and balls', 'Remainder']].apply(lambda x: np.log(x+1))
    print(data_norm.describe())
    data_norm.reset_index(drop=True)
    data_norm.insert(loc=0, column='Row ID', value=data['Row ID'])
    data_norm.insert(loc=2, column='Remaining Team Value', value=data['Remaining Team Value'])
    data_norm.insert(loc=7, column='Winner_num', value=data['Winner_num'])
    plt.show()
    return data_norm

In [ ]:
def corr(data, type):
    corr_matrix = data.corr(method = type)
    sns.set(rc={'figure.figsize':(12,10)})
    new_cmap = sns.diverging_palette(120, 120, as_cmap=True)
    sns.heatmap(corr_matrix, annot=True, cmap=new_cmap, vmin=-1, vmax=1)
    return corr_matrix

Skewing doesn't affect tree-based methods so should I do this or not?

In [ ]:
#Can't use boxcox because the values must be strictly positive, i.e. not negative or 0
#(data_train.values < 0).any()
#data_train[data_train['Wickets taken']==0]

#data_train_flat = data_train.values.flatten()
#data_train_flat


#transformed_data, best_lambda = stats.boxcox(data_train_flat) 
#sns.distplot(transformed_data, hist=False, kde=True) 

In [ ]:
data_train_lognorm_df = normal(data_train)
data_train_lognorm_SMOTEmin_df = normal(data_train_SMOTEmin)
data_train_lognorm_SMOTEauto_df = normal(data_train_SMOTEauto)

In [ ]:
data_train_lognorm_df

In [ ]:
corr(data_train_lognorm_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_lognorm_SMOTEmin_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_lognorm_SMOTEauto_df, 'pearson')['Winner_num']

Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [ ]:
def scaling(data, scaler):
    scale_data = data.drop(['Winner_num', 'Row ID'], axis =1)

    if scaler == MinMaxScaler:
        min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
        data_scaled = min_max_scaler.fit_transform(scale_data)

    elif scaler == StandardScaler:
        std_scaler = StandardScaler()
        data_scaled = std_scaler.fit_transform(scale_data)

    data_scaled = pd.DataFrame(data_scaled)
    data_scaled.rename(columns={0:'Wickets taken', 1: 'Remaining Team Value', 2: 'Remaining Average Team Value', 3: 'Start Team Value', 4: 'Ground RPO', 5: 'Ground RPW', 6:'Remaining overs and balls', 7:'Remainder'}, inplace=True)
    data_scaled.reset_index(drop=True)
    data_scaled.insert(loc=0, column='Row ID', value=data['Row ID'])
    data_scaled.insert(loc=7, column='Winner_num', value=data['Winner_num'])
    return data_scaled

In [ ]:
scaling(data_train_lognorm_df, StandardScaler)

In [ ]:
#Non smote
data_train_min_max_scaled_df = scaling(data_train_lognorm_df, MinMaxScaler)
data_train_standard_scaled_df = scaling(data_train_lognorm_df, StandardScaler)

#Smote min
data_train_min_max_scaled_SMOTEmin_df = scaling(data_train_lognorm_SMOTEmin_df, MinMaxScaler)
data_train_standard_scaled_SMOTEmin_df = scaling(data_train_lognorm_SMOTEmin_df, StandardScaler)

#Smote auto
data_train_min_max_scaled_SMOTEauto_df = scaling(data_train_lognorm_SMOTEauto_df, MinMaxScaler)
data_train_standard_scaled_SMOTEauto_df = scaling(data_train_lognorm_SMOTEauto_df, StandardScaler)

In [ ]:
corr(data_train_min_max_scaled_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_standard_scaled_df, 'pearson')['Winner_num']

Clustering for new attributes?


In [ ]:
corr(data_train_min_max_scaled_SMOTEmin_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_standard_scaled_SMOTEmin_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_min_max_scaled_SMOTEauto_df, 'pearson')['Winner_num']

In [ ]:
corr(data_train_standard_scaled_SMOTEauto_df, 'pearson')['Winner_num']

In [ ]:
data_train_min_max_scaled_df.to_csv("data_train_min_max_scaled_csv")
data_train_standard_scaled_df.to_csv("data_train_standard_scaled_csv")

data_train_min_max_scaled_SMOTEmin_df.to_csv("data_train_min_max_scaled_SMOTEmin_csv")
data_train_standard_scaled_SMOTEmin_df.to_csv("data_train_standard_scaled_SMOTEmin_csv")

data_train_min_max_scaled_SMOTEauto_df.to_csv("data_train_min_max_scaled_SMOTEauto_csv")
data_train_standard_scaled_SMOTEauto_df.to_csv("data_train_standard_scaled_SMOTEauto_csv")

data_test.to_csv("data_test_csv")

Experiment with Attribute combinations

Commentary

Shall I plot the Winner before I categorise?
Very imbalanced - 1 and 2 mainly fine but 0 not.

Be careful that any data added is to the end, given the row index is a unique iD.

https://www.analyticsvidhya.com/blog/2020/07/10-techniques-to-deal-with-class-imbalance-in-machine-learning/#:~:text=In%20a%20dataset%20with%20highly,obviously%20not%20the%20correct%20one.

Models

Random Forest
SVM
Neural Network

Questions:
Should I change the 0s to 3s or similar for Winner_num so that I don't have to add things to them to do the lognorm etc.